In [2]:
import pandas as pd
from geopandas import GeoDataFrame
from fiona.crs import from_epsg
import shapely
import matplotlib as mlt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import csv

In [4]:
bus_brk_calendar = pd.DataFrame.from_csv("google_bus_brooklyn/calendar.txt")

In [5]:
bus_brk_calendar.head(3)

,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
service_id,,,,,,,,,
EN_A7-Sunday,0,0,0,0,0,0,1,20170108,20170326
EN_A7-Weekday-SDon,1,1,1,1,1,0,0,20170109,20170331
EN_A7-Saturday,0,0,0,0,0,1,0,20170114,20170401


In [6]:
bus_brk_routes = pd.DataFrame.from_csv("google_bus_brooklyn/routes.txt")

In [7]:
bus_brk_routes.head(3)

,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
route_id,,,,,,,,
B1,MTA NYCT,B1,Bay Ridge - Manhattan Beach,via 86th St / Ocean Pkwy,3,http://web.mta.info/nyct/bus/schedule/bkln/b00...,00AEEF,FFFFFF
B11,MTA NYCT,B11,Sunset Park - Midwood,via 49th & 50th St / Avenue J,3,http://web.mta.info/nyct/bus/schedule/bkln/b01...,006CB7,FFFFFF
B12,MTA NYCT,B12,Lefferts Gardens - East New York,via Clarkson Av / Empire Blvd / East New York Av,3,http://web.mta.info/nyct/bus/schedule/bkln/b01...,6CBE45,FFFFFF


In [8]:
bus_brk_stop_t = pd.DataFrame.from_csv("google_bus_brooklyn/stop_times.txt")

In [11]:
def early_late(reader):
    early = {}
    late = {}
    for row in reader:
        
        t = row["arrival_time"]
        t = int(t[0:2]) *60 + int(t[3:5])   # because 25:00:00, as late night rather early
        stop = row["stop_id"]
        
        
        if stop in late.keys():
            compare_l = late[stop]['arrival_time']
            compare_l = int(compare_l[0:2]) *60 + int(compare_l[3:5])
            if t > compare_l :
                late[stop] = row
            else:
                pass
        else:
            late[stop] = row
         
        
        
        if stop in early.keys():
            compare_e = early[stop]['arrival_time']
            compare_e = int(compare_e[0:2]) *60 + int(compare_e[3:5])
            if t < compare_e :
                early[stop] = row
            else:
                pass
        else:
            early[stop] = row
        
    return early,late
    
### NOTE: You SHOULD NOT modify the code below. If you
### write your firstRide generator above correctly, the
### code below will output the correct information

#### for each bus in manhatten, 'early' is the start of the working time, late is the end of working time

In [12]:
with open("google_bus_manhattan/stop_times.txt", 'r') as fi:
    reader = csv.DictReader(fi)
    early,late = early_late(reader)

In [40]:
bus_brk_stop = pd.DataFrame.from_csv("google_bus_brooklyn/stops.txt")

In [31]:
del bus_brk_stop['stop_desc']
del bus_brk_stop['zone_id']
del bus_brk_stop['stop_url']
del bus_brk_stop['parent_station']
del bus_brk_stop['location_type']

In [32]:
bus_brk_stop = bus_brk_stop.reset_index()

In [33]:
df_early = pd.DataFrame.from_dict(early).T
df_late = pd.DataFrame.from_dict(late).T
df_early = df_early[['arrival_time','stop_id','trip_id']]
df_late = df_late[['arrival_time','stop_id','trip_id']]
df_early = df_early.rename(columns= {"arrival_time" :"earliest","trip_id":"earliest_trip_id"})
df_late = df_late.rename(columns= {"arrival_time" :"latest","trip_id":"latest_trip_id"})
df_early['stop_id'] = pd.to_numeric(df_early['stop_id'])
df_late['stop_id'] = pd.to_numeric(df_late['stop_id'])
bus_brk_stop = bus_brk_stop.merge(df_early,on="stop_id")
bus_brk_stop = bus_brk_stop.merge(df_late,on="stop_id")

In [34]:
bus_brk_stop

,stop_id,stop_name,stop_lat,stop_lon,earliest,earliest_trip_id,latest,latest_trip_id
0,300000,ORIENTAL BL/MACKENZIE ST,40.578251,-73.939743,01:00:00,UP_H7-Weekday-006000_B1_1,24:25:00,UP_A7-Saturday-146500_B1_35
1,300002,ORIENTAL BL/JAFFRAY ST,40.578068,-73.943031,01:00:38,UP_H7-Weekday-006000_B1_1,26:55:40,FB_A7-Sunday-161500_B49_22
2,300003,ORIENTAL BL/HASTINGS ST,40.577961,-73.944664,01:00:56,UP_H7-Weekday-006000_B1_1,26:55:59,FB_A7-Sunday-161500_B49_22
3,300004,ORIENTAL BL/FALMOUTH ST,40.577721,-73.946205,01:01:13,UP_H7-Weekday-006000_B1_1,26:56:16,FB_A7-Sunday-161500_B49_22
4,300006,ORIENTAL BL/DOVER ST,40.577354,-73.949554,01:01:50,UP_H7-Weekday-006000_B1_1,26:56:54,FB_A7-Sunday-161500_B49_22
5,300007,ORIENTAL BL/BEAUMONT ST,40.577148,-73.951385,01:02:11,UP_H7-Weekday-006000_B1_1,26:57:15,FB_A7-Sunday-161500_B49_22
6,300008,ORIENTAL BL/WEST END AV,40.577003,-73.952721,01:02:26,UP_H7-Weekday-006000_B1_1,26:57:30,FB_A7-Sunday-161500_B49_22
7,300009,BRIGHTON BEACH AV/CORBIN PL,40.577461,-73.954407,01:02:53,UP_H7-Weekday-006000_B1_1,24:28:36,UP_A7-Saturday-146500_B1_35
8,300010,BRIGHTON BEACH AV/BRIGHTON 14 ST,40.577957,-73.956383,01:03:17,UP_H7-Weekday-006000_B1_1,24:29:06,UP_A7-Saturday-146500_B1_35
9,300011,BRIGHTON BEACH AV/BRIGHTON 12 ST,40.578369,-73.958176,01:03:37,UP_H7-Weekday-006000_B1_1,24:29:31,UP_A7-Saturday-146500_B1_35


In [36]:
bus_brk_stop.to_csv("bus_brk_stop")

In [41]:
bus_mht_stop = pd.DataFrame.from_csv("google_bus_manhattan/stops.txt")

del bus_mht_stop['stop_desc']
del bus_mht_stop['zone_id']
del bus_mht_stop['stop_url']
del bus_mht_stop['parent_station']
del bus_mht_stop['location_type']

bus_mht_stop = bus_mht_stop.reset_index()

df_early = pd.DataFrame.from_dict(early).T
df_late = pd.DataFrame.from_dict(late).T
df_early = df_early[['arrival_time','stop_id','trip_id']]
df_late = df_late[['arrival_time','stop_id','trip_id']]
df_early = df_early.rename(columns= {"arrival_time" :"earliest","trip_id":"earliest_trip_id"})
df_late = df_late.rename(columns= {"arrival_time" :"latest","trip_id":"latest_trip_id"})
df_early['stop_id'] = pd.to_numeric(df_early['stop_id'])
df_late['stop_id'] = pd.to_numeric(df_late['stop_id'])
bus_mht_stop = bus_mht_stop.merge(df_early,on="stop_id")
bus_mht_stop = bus_mht_stop.merge(df_late,on="stop_id")

In [42]:
bus_mht_stop

,stop_id,stop_name,stop_lat,stop_lon,earliest,earliest_trip_id,latest,latest_trip_id
0,400001,4 AV/E 9 ST,40.730640,-73.990440,02:00:00,MV_A7-Saturday-012000_M2_101,25:20:00,MV_A7-Saturday-152000_M2_142
1,400002,4 AV/E 12 ST,40.732609,-73.989960,02:00:46,MV_A7-Saturday-012000_M2_101,25:20:46,MV_A7-Saturday-152000_M2_142
2,400003,4 AV/E 13 ST,40.733936,-73.989723,02:01:17,MV_A7-Saturday-012000_M2_101,25:21:17,MV_A7-Saturday-152000_M2_142
3,400007,PARK AV S/E 21 ST,40.738976,-73.987053,02:03:28,MV_A7-Saturday-012000_M2_101,25:23:28,MV_A7-Saturday-152000_M2_142
4,400026,MADISON AV/E 63 ST,40.766273,-73.969330,02:14:08,MV_A7-Saturday-012000_M2_101,25:34:08,MV_A7-Saturday-152000_M2_142
5,400027,MADISON AV/E 65 ST,40.767582,-73.968376,05:06:00,MQ_A7-Weekday-030000_M66_2,25:28:00,MQ_H7-Weekday-152000_M66_24
6,400028,MADISON AV/E 67 ST,40.768814,-73.967484,02:14:59,MV_A7-Saturday-012000_M2_101,25:34:59,MV_A7-Saturday-152000_M2_142
7,400029,MADISON AV/E 69 ST,40.770077,-73.966560,02:15:24,MV_A7-Saturday-012000_M2_101,25:35:24,MV_A7-Saturday-152000_M2_142
8,400030,MADISON AV/E 72 ST,40.772148,-73.965050,02:16:05,MV_A7-Saturday-012000_M2_101,25:36:05,MV_A7-Saturday-152000_M2_142
9,400032,MADISON AV/E 77 ST,40.775322,-73.962738,02:17:08,MV_A7-Saturday-012000_M2_101,25:37:08,MV_A7-Saturday-152000_M2_142


In [43]:
bus_brk_stop.to_csv("bus_mht_stop")

In [47]:
df = {}
boro = ['brooklyn','manhattan','queens','bronx','staten_island']
for i in boro:
    docu = "google_bus_" + i +'/stops.txt'
    df[i] = pd.DataFrame.from_csv(docu, index_col= False)
    df[i] = df[i][["stop_id","stop_name",'stop_lat',"stop_lon"]]

In [54]:
frames = [df[i] for i in boro]

In [58]:
bus_stops = pd.concat(frames, keys=boro)

In [59]:
bus_stops

stop_id                          stop_name   stop_lat  \
brooklyn      0      300000           ORIENTAL BL/MACKENZIE ST  40.578251   
              1      300002             ORIENTAL BL/JAFFRAY ST  40.578068   
              2      300003            ORIENTAL BL/HASTINGS ST  40.577961   
              3      300004            ORIENTAL BL/FALMOUTH ST  40.577721   
              4      300006               ORIENTAL BL/DOVER ST  40.577354   
              5      300007            ORIENTAL BL/BEAUMONT ST  40.577148   
              6      300008            ORIENTAL BL/WEST END AV  40.577003   
              7      300009        BRIGHTON BEACH AV/CORBIN PL  40.577461   
              8      300010   BRIGHTON BEACH AV/BRIGHTON 14 ST  40.577957   
              9      300011   BRIGHTON BEACH AV/BRIGHTON 12 ST  40.578369   
              10     300012  BRIGHTON BEACH AV/CONEY ISLAND AV  40.577900   
              11     300013    BRIGHTON BEACH AV/BRIGHTON 5 ST  40.577404   
              12     300014    BRIGHTON BEACH AV/BRIGHTON 3 ST  40.577034   
              13     300016         BRIGHTON BEACH AV/OCEAN PY  40.576424   
              14     300017             OCEAN PY/OCEAN VIEW AV  40.578739   
              15     300018                OCEAN PY/NEPTUNE AV  40.580219   
              16     300019                   OCEAN PY/WEST AV  40.581417   
              17     300020                  OCEAN PY/SHORE PY  40.585468   
              18     300022                        AV X/E 2 ST  40.590771   
              19     300023                       AV X/WEST ST  40.590565   
              20     300024                        AV X/W 2 ST  40.590343   
              21     300025                  AV X/MC DONALD AV  40.590191   
              22     300026               86 ST/VAN SICKLEN ST  40.591331   
              23     300028                       86 ST/W 9 ST  40.593826   
              24     300029                      86 ST/W 11 ST  40.595116   
              25     300030                         86 ST/AV U  40.595741   
              26     300031                 86 ST/STILLWELL AV  40.596752   
              27     300032                        86 ST/25 AV  40.597874   
              28     300033                        86 ST/24 AV  40.599205   
              29     300034                        86 ST/23 AV  40.600525   
...                     ...                                ...        ...   
staten_island 2048   905114        WEST SHORE PLAZA/CHELSEA ST  40.601971   
              2049   905121        bayonne br/bayonne br-north  40.644947   
              2050   905122        bayonne br/bayonne br-south  40.645042   
              2051   905130    ARTHUR KILL RD/CHARLESTON DEPOT  40.532368   
              2052   905140                   NEW DORP LA/3 ST  40.575218   
              2053   905141            RICHMOND RD/NEW DORP LA  40.575829   
              2054   905142                         4 AV/87 ST  40.621979   
              2055   905144               MARSH AV/WESTPORT ST  40.584103   
              2056   905148                  west st/morris st  40.706863   
              2057   905149             murray st/north end av  40.715824   
              2058   905150                  west st/morris st  40.706837   
              2059   905153       staten island expy/wooley av  40.607510   
              2060   905154      TELEPORT DR/CORP COMMONS BLDG  40.604549   
              2061   905155      TELEPORT DR/CORP COMMONS BLDG  40.604706   
              2062   905156            TELEPORT DR/SOUTH AV #1  40.606880   
              2063   905157            TELEPORT DR/SOUTH AV #2  40.607052   
              2064   905160         N.J. Tpke/route 439 (N.B.)  40.643555   
              2065   905161                   MARSH AV/RING RD  40.583721   
              2066   905162                   MARSH AV/RING RD  40.583744   
              2067   905167                   MARSH AV/RING RD  40.583691   
              20

In [61]:
point_1_lat = (40.657-0.005, 40.657 +0.005)
point_1_lon = (-73.987-0.005, -73.987+0.005)

In [70]:
bus_stops.ix['brooklyn']

,stop_id,stop_name,stop_lat,stop_lon
0,300000,ORIENTAL BL/MACKENZIE ST,40.578251,-73.939743
1,300002,ORIENTAL BL/JAFFRAY ST,40.578068,-73.943031
2,300003,ORIENTAL BL/HASTINGS ST,40.577961,-73.944664
3,300004,ORIENTAL BL/FALMOUTH ST,40.577721,-73.946205
4,300006,ORIENTAL BL/DOVER ST,40.577354,-73.949554
5,300007,ORIENTAL BL/BEAUMONT ST,40.577148,-73.951385
6,300008,ORIENTAL BL/WEST END AV,40.577003,-73.952721
7,300009,BRIGHTON BEACH AV/CORBIN PL,40.577461,-73.954407
8,300010,BRIGHTON BEACH AV/BRIGHTON 14 ST,40.577957,-73.956383
9,300011,BRIGHTON BEACH AV/BRIGHTON 12 ST,40.578369,-73.958176


In [76]:
bus_stops.ix['brooklyn']

,stop_id,stop_name,stop_lat,stop_lon
0,300000,ORIENTAL BL/MACKENZIE ST,40.578251,-73.939743
1,300002,ORIENTAL BL/JAFFRAY ST,40.578068,-73.943031
2,300003,ORIENTAL BL/HASTINGS ST,40.577961,-73.944664
3,300004,ORIENTAL BL/FALMOUTH ST,40.577721,-73.946205
4,300006,ORIENTAL BL/DOVER ST,40.577354,-73.949554
5,300007,ORIENTAL BL/BEAUMONT ST,40.577148,-73.951385
6,300008,ORIENTAL BL/WEST END AV,40.577003,-73.952721
7,300009,BRIGHTON BEACH AV/CORBIN PL,40.577461,-73.954407
8,300010,BRIGHTON BEACH AV/BRIGHTON 14 ST,40.577957,-73.956383
9,300011,BRIGHTON BEACH AV/BRIGHTON 12 ST,40.578369,-73.958176


In [81]:
point_1_lat
point_1_lon

(-73.99199999999999, -73.982)

In [78]:
for i in range(len(bus_stops.ix['brooklyn'])):
    stop_lat = bus_stops.ix['brooklyn',i]['stop_lat']
    stop_lon = bus_stops.ix['brooklyn',i]['stop_lon']
    if stop_lat<= point_1_lat[1] and stop_lat >= point_1_lat[0]:
        if stop_lon<= point_1_lon[1] and stop_lat >= point_1_lon[0]:
            print  bus_stops.ix['brooklyn',i]['stop_id']
        
    

302678
302679
302680
302782
302783
302784
302898
302899
302900
302903
302905
302971
302973
302975
302976
305364
305365
305367
305369
305370
305431
305432
305434
305437
305624
305626
305681
305682
305858
305906
305907
305909
306421
306556
306571
306573
306890
307047
307177
307227
307521
307709
308084
308194
308331
308335
308336
801136
901464
901481
901488


In [66]:
bus_stops.iloc[0]

stop_id                        300000
stop_name    ORIENTAL BL/MACKENZIE ST
stop_lat                      40.5783
stop_lon                     -73.9397
Name: (brooklyn, 0), dtype: object

In [60]:
bus_stops.to_csv("bus_stops.csv")